In [1]:
from os.path import realpath
import glob
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import join as join
import seaborn as sns
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import BatchNormalization

from keras.applications.vgg16 import VGG16

import os

2022-10-23 22:46:43.046678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 22:46:43.157066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:
2022-10-23 22:46:43.157083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-23 22:46:43.181671: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regi

In [2]:
activation = "sigmoid"

In [3]:
SIZE = 256

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

In [5]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(SIZE, 
                                                              SIZE,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
    
  ]
)

2022-10-23 03:39:25.045687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-23 03:39:25.045893: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:
2022-10-23 03:39:25.045952: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.10/dist-packages/cv2/../../lib64:
2022-10-23 03:39:25.046007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: ca

In [21]:

from tensorflow import optimizers

In [45]:
from keras.layers import Dropout


feature_extractor = Sequential()
feature_extractor.add(data_augmentation)
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (SIZE, SIZE, 3)))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())
feature_extractor.add(Dropout(0.1))
feature_extractor.add(Flatten())


x = feature_extractor.output  
x = Dense(256, activation = activation, kernel_initializer = 'he_uniform')(x)
prediction_layer = Dense(6, activation = 'softmax')(x)
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer="rmsprop",loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary()) 

NameError: name 'data_augmentation' is not defined

In [5]:
df_interno = pd.read_csv('./df_interno.csv')

In [6]:
df_interno.replace({"CLASSE":[6, 7]}, 5, inplace=True)

In [7]:
df_interno

,CLASSE,VIGOR,0,1,2,3,4,5,6,7,...,196598,196599,196600,196601,196602,196603,196604,196605,196606,196607
0,4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.360784,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.4,0.360784,0.407843,0.490196,0.466667,0.509804
3,4,1,0.000000,0.000000,0.000000,0.321569,0.286275,0.333333,0.313725,0.282353,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,4,1,0.266667,0.231373,0.282353,0.250980,0.211765,0.262745,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
446,4,1,0.301961,0.282353,0.317647,0.321569,0.286275,0.337255,0.305882,0.254902,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
447,4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
448,4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.329412,0.278431,0.341176


In [8]:
vigor = df_interno['VIGOR']

In [9]:
classe = df_interno['CLASSE']

In [10]:
classe.unique()

array([4, 2, 3, 5, 1])

In [11]:
df_interno.drop(['CLASSE', 'VIGOR'], axis = 1, inplace=True)

In [12]:
df_interno.shape

(450, 196608)

In [13]:
df_interno = df_interno.to_numpy()

In [17]:
df_interno = np.reshape(df_interno, (450, 256, 256, 3))

In [18]:
classe = classe.to_numpy()

In [19]:
from sklearn.ensemble import RandomForestClassifier  #Random Forest algorithm
from sklearn.model_selection import train_test_split

In [20]:
from keras.utils import to_categorical



In [21]:
X_train, X_test, y_train, y_test = train_test_split(df_interno, classe, test_size=0.25)

In [22]:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [23]:
y_test_one_hot.shape

(113, 6)

In [24]:
y_train_one_hot.shape

(337, 6)

In [22]:

history = cnn_model.fit(X_train, y_train_one_hot, epochs=1000, validation_data = (X_test, y_test_one_hot))

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [53]:
X_test.shape

(113, 128, 128, 3)

In [55]:
y_train_one_hot.shape

(337, 6)

In [57]:
y_test_one_hot.shape

(113, 6)

In [35]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet101(include_top=False,
                   input_shape=(128,128,3),
                   pooling='avg',classes=6,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=True

resnet_model.add(pretrained_model)

resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(6, activation='softmax'))

In [43]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(128,128,3),
                   pooling='avg',classes=6,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=True

resnet_model.add(pretrained_model)

resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(6, activation='softmax'))

In [31]:
from tensorflow.keras.optimizers import Adam


In [68]:
resnet_model.compile(optimizer="rmsprop",loss='categorical_crossentropy',metrics=['accuracy'])

history = resnet_model.fit(X_train, y_train_one_hot, validation_data=(X_test, y_test_one_hot), epochs=100)

Epoch 1/100
11/11 [==============================] - 40s 4s/step - loss: 10.6573 - accuracy: 0.5371 - val_loss: 1.1009 - val_accuracy: 0.6460
Epoch 2/100
11/11 [==============================] - 38s 3s/step - loss: 1.1552 - accuracy: 0.5875 - val_loss: 1.0629 - val_accuracy: 0.6460
Epoch 3/100
11/11 [==============================] - 39s 4s/step - loss: 1.1468 - accuracy: 0.5875 - val_loss: 1.0711 - val_accuracy: 0.6460
Epoch 4/100
11/11 [==============================] - 39s 4s/step - loss: 1.1427 - accuracy: 0.5875 - val_loss: 1.0810 - val_accuracy: 0.6460
Epoch 5/100
11/11 [==============================] - 39s 4s/step - loss: 1.1367 - accuracy: 0.5875 - val_loss: 1.2019 - val_accuracy: 0.6460
Epoch 6/100
11/11 [==============================] - 39s 3s/step - loss: 1.1608 - accuracy: 0.5875 - val_loss: 1.1383 - val_accuracy: 0.6460
Epoch 7/100
11/11 [==============================] - 39s 4s/step - loss: 1.1414 - accuracy: 0.5875 - val_loss: 1.1005 - val_accuracy: 0.6460
Epoch 8/100


In [45]:
from classificadores import *

In [25]:
from sklearn import naive_bayes, svm


VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0

#Now, let us use features from convolutional network for RF
feature_extractor=VGG_model.predict(X_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_RF = features #This is our X input to RF

RF_model = RandomForestClassifier(n_estimators = 250, max_depth=10, random_state = 42)

# Train the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding

#Send test data through same feature extractor process
X_test_feature = VGG_model.predict(X_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

#Now predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_features)


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

2022-10-23 11:48:22.506471: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2022-10-23 11:48:22.689183: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.


 1/11 [=>............................] - ETA: 32s

2022-10-23 11:48:25.404195: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.
2022-10-23 11:48:25.499239: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.


 2/11 [====>.........................] - ETA: 25s

2022-10-23 11:48:28.286823: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 536870912 exceeds 10% of free system memory.


4/4 [==============================] - 10s 2s/step


In [ ]:
cross_val_score

In [67]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=5000,
    random_seed=42,
    learning_rate=0.001,
    custom_loss=['AUC', 'Accuracy'],
#   verbose=5,
    task_type="GPU"
)

clf.fit(
    X_for_RF, y_train,
    eval_set=(X_test_features, y_test),
)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 1.6086016	test: 1.6086412	best: 1.6086412 (0)	total: 134ms	remaining: 11m 11s
1:	total: 274ms	remaining: 11m 24s
2:	total: 389ms	remaining: 10m 47s
3:	total: 509ms	remaining: 10m 35s
4:	total: 630ms	remaining: 10m 29s
5:	learn: 1.6046494	test: 1.6057186	best: 1.6057186 (5)	total: 757ms	remaining: 10m 30s
6:	total: 871ms	remaining: 10m 21s
7:	total: 1.01s	remaining: 10m 29s
8:	total: 1.12s	remaining: 10m 19s
9:	total: 1.23s	remaining: 10m 12s
10:	learn: 1.6005396	test: 1.6021639	best: 1.6021639 (10)	total: 1.35s	remaining: 10m 14s
11:	total: 1.48s	remaining: 10m 17s
12:	total: 1.62s	remaining: 10m 21s
13:	total: 1.75s	remaining: 10m 22s
14:	total: 1.87s	remaining: 10m 23s
15:	learn: 1.5965207	test: 1.5989212	best: 1.5989212 (15)	total: 1.99s	remaining: 10m 20s
16:	total: 2.09s	remaining: 10m 13s
17:	total: 2.22s	remaining: 10m 15s
18:	total: 2.35s	remaining: 10m 16s
19:	total: 2.48s	remaining: 10m 16s
20:	learn: 1.5925405	test: 1.5958519	best: 1.5958519 (20)	total: 2.61s	remai

In [28]:
prediction_RF = clf.predict(X_test_features)

NameError: name 'clf' is not defined

In [27]:

from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_RF))

Accuracy =  0.5486725663716814


In [38]:
from sklearn import svm

In [41]:
feature_extractor = resnet_model.predict(X_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_RF = features #This is our X input to RF

RF_model = RandomForestClassifier(n_estimators = 250, max_depth=10, random_state = 42)

# Train the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding

#Send test data through same feature extractor process
X_test_feature = resnet_model.predict(X_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

#Now predict using the trained RF model. 
prediction_RF = RF_model.predict(X_test_features)

4/4 [==============================] - 2s 471ms/step


In [15]:

from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test, prediction_RF))

NameError: name 'y_test' is not defined

In [35]:
X_for_RF = feature_extractor.predict(df_interno)

AttributeError: 'numpy.ndarray' object has no attribute 'predict'

In [ ]:
#adaBoostClassificator(X_for_RF, classe, 5)

KeyboardInterrupt: 